<a href="https://colab.research.google.com/github/yomna-nasr/barcode-food-scanner/blob/main/Food_Scanner_%2B_Personal_Health_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run: !pip install requests rich pandas ipywidgets
!pip install requests rich pandas ipywidgets

In [ ]:
# 🔍 Food Scanner + Personal Health Filter (Colab version)
# ------------------------------------------------------------

import re
import requests
from rich import print, box as rbox
from rich.table import Table
from ipywidgets import widgets, Layout
from IPython.display import display, clear_output

API = "https://world.openfoodfacts.org/api/v0/product/{}.json"

# ------------------------------------------------------------------
# Helper functions
# ------------------------------------------------------------------
def to_float(x):
    """Convert various formats to float, return None if impossible."""
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    cleaned = re.sub(r"[^0-9eE\.\,\+\-]", "", str(x)).replace(",", ".")
    try:
        return float(cleaned)
    except ValueError:
        return None

def get_serving_size(prod, default=100.0):
    """Extract serving size from product data."""
    if prod.get("serving_quantity") is not None:
        v = to_float(prod["serving_quantity"])
        if v:
            return v
    if prod.get("serving_size"):
        m = re.search(r"([\d\.,]+)\s*(g|ml)", prod["serving_size"], re.I)
        if m:
            return to_float(m.group(1))
    return default

def fetch_product(code):
    """Fetch product from OpenFoodFacts API."""
    r = requests.get(API.format(code), timeout=10)
    if r.status_code != 200:
        raise RuntimeError(f"HTTP error {r.status_code}")
    data = r.json()
    if data.get("status") != 1:
        raise ValueError("Product not found.")
    return data["product"]

# ------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------
QUESTIONS = {
    "diabetes":       "Diabetes",
    "cholesterol":    "High cholesterol",
    "blood_pressure": "High blood pressure",
    "heart":          "Heart condition",
    "weight_goal":    "Weight goal",
    "activity":       "Activity level",
}

# Baseline daily caps (realistic adult guidelines)
BASE_CAPS = {
    "sugars":        50,      # WHO: <50g/day (10% of 2000 kcal)
    "carbohydrates": 300,     # ~60% of 2000 kcal
    "fat":           65,      # ~30% of 2000 kcal
    "saturated-fat": 20,      # <10% of 2000 kcal
    "fiber":         30,      # Daily recommendation
    "proteins":      50,      # ~10% of 2000 kcal
    "salt":          5,       # WHO: <5g/day
    "sodium":        2.0,     # WHO: <2g/day (5g salt)
    "cholesterol":   0.3,     # 300mg/day
    "energy-kcal":   2000,    # Average adult
}

# Nutrients that "belong" to each condition
FOCUS = {
    "diabetes":       ["sugars", "carbohydrates"],
    "cholesterol":    ["saturated-fat", "fat", "cholesterol"],
    "blood_pressure": ["salt", "sodium"],
    "heart":          ["saturated-fat", "salt", "cholesterol"],
}

# Severity multipliers: more realistic adjustments
# 0 → normal, 1 → moderately strict (-25%), 2 → very strict (-50%)
SEVERITY_FACTOR = {0: 1.00, 1: 0.75, 2: 0.50}

# ------------------------------------------------------------------
# Create input widgets
# ------------------------------------------------------------------
print("Health Profile Setup")
print("=" * 50)

# Personal info
weight_input = widgets.FloatText(
    value=70.0,
    description='Weight (kg):',
    style={'description_width': '200px'},
    layout=Layout(width='300px')
)

height_input = widgets.FloatText(
    value=170.0,
    description='Height (cm):',
    style={'description_width': '200px'},
    layout=Layout(width='300px')
)

age_input = widgets.IntText(
    value=30,
    description='Age (years):',
    style={'description_width': '200px'},
    layout=Layout(width='300px')
)

gender_input = widgets.RadioButtons(
    options=['Male', 'Female'],
    value='Male',
    description='Gender:',
    style={'description_width': '200px'},
    layout=Layout(width='400px')
)

# Create radio buttons for each condition
widgets_dict = {}
for key, label in QUESTIONS.items():
    if key == "weight_goal":
        widgets_dict[key] = widgets.RadioButtons(
            options=['Lose Weight (0)', 'Maintain (1)', 'Gain Weight (2)'],
            value='Maintain (1)',
            description=label + ':',
            style={'description_width': '200px'},
            layout=Layout(width='500px')
        )
    elif key == "activity":
        widgets_dict[key] = widgets.RadioButtons(
            options=['Sedentary (0)', 'Moderate (1)', 'Very Active (2)'],
            value='Moderate (1)',
            description=label + ':',
            style={'description_width': '200px'},
            layout=Layout(width='500px')
        )
    else:
        widgets_dict[key] = widgets.RadioButtons(
            options=['None (0)', 'Mild/Controlled (1)', 'Severe/Uncontrolled (2)'],
            value='None (0)',
            description=label + ':',
            style={'description_width': '200px'},
            layout=Layout(width='500px')
        )

# Servings input
servings_input = widgets.IntText(
    value=1,
    description='Servings:',
    style={'description_width': '200px'},
    layout=Layout(width='300px')
)

# Barcode input
barcode_input = widgets.Text(
    value='',
    placeholder='Enter barcode...',
    description='Barcode:',
    style={'description_width': '200px'},
    layout=Layout(width='500px')
)

# Submit button
submit_btn = widgets.Button(
    description='Analyze Product',
    button_style='success',
    layout=Layout(width='200px')
)

# Output area
output = widgets.Output()

# Display all widgets
print("\n--- Personal Information ---")
display(weight_input)
display(height_input)
display(age_input)
display(gender_input)

print("\n--- Health & Lifestyle ---")
for w in widgets_dict.values():
    display(w)
display(servings_input)
display(barcode_input)
display(submit_btn)
display(output)

# ------------------------------------------------------------------
# Button click handler
# ------------------------------------------------------------------
def on_submit_clicked(b):
    with output:
        clear_output()

        # Get personal info
        weight = weight_input.value
        height = height_input.value
        age = age_input.value
        gender = gender_input.value

        if weight <= 0 or height <= 0 or age <= 0:
            print("[bold red]Please enter valid personal information![/]")
            return

        # Calculate BMI
        bmi = weight / ((height / 100) ** 2)

        # Calculate BMR (Mifflin-St Jeor Equation)
        if gender == 'Male':
            bmr = (10 * weight) + (6.25 * height) - (5 * age) + 5
        else:
            bmr = (10 * weight) + (6.25 * height) - (5 * age) - 161

        # Get profile values
        profile = {}
        for key, widget in widgets_dict.items():
            val = widget.value
            if '0' in val or 'Sedentary' in val or 'Lose' in val:
                profile[key] = 0
            elif '1' in val or 'Moderate' in val or 'Maintain' in val:
                profile[key] = 1
            else:
                profile[key] = 2

        # Calculate TDEE based on activity level
        activity_multiplier = {0: 1.2, 1: 1.55, 2: 1.9}  # Sedentary, Moderate, Very Active
        tdee = bmr * activity_multiplier[profile['activity']]

        # Adjust calories based on weight goal
        if profile['weight_goal'] == 0:  # Lose weight
            daily_calories = tdee - 500  # 500 calorie deficit
            calorie_advice = "weight loss"
        elif profile['weight_goal'] == 2:  # Gain weight
            daily_calories = tdee + 300  # 300 calorie surplus
            calorie_advice = "weight gain"
        else:  # Maintain
            daily_calories = tdee
            calorie_advice = "maintenance"

        profile["servings"] = servings_input.value
        barcode = barcode_input.value.strip()

        if not barcode:
            print("[bold red]Please enter a barcode![/]")
            return

        if profile["servings"] < 1:
            print("[bold red]Servings must be at least 1![/]")
            return

        # BMI category
        if bmi < 18.5:
            bmi_cat = "Underweight"
            bmi_color = "yellow"
        elif bmi < 25:
            bmi_cat = "Normal"
            bmi_color = "green"
        elif bmi < 30:
            bmi_cat = "Overweight"
            bmi_color = "yellow"
        else:
            bmi_cat = "Obese"
            bmi_color = "red"

        # Display profile
        print("\n[bold cyan]📊 YOUR PROFILE[/]")
        print(f" • Weight: {weight}kg | Height: {height}cm | Age: {age} | Gender: {gender}")
        print(f" • BMI: [{bmi_color}]{bmi:.1f}[/{bmi_color}] ({bmi_cat})")
        print(f" • Daily calories ({calorie_advice}): {daily_calories:.0f} kcal")
        print(f" • Servings to consume: {profile['servings']}")

        print("\n[bold cyan]🏥 HEALTH CONDITIONS[/]")
        levels = {0: "none", 1: "mild", 2: "severe"}
        for k, v in profile.items():
            if k in ["servings", "weight_goal", "activity"]:
                continue
            print(f" • {QUESTIONS[k]}: {levels[v]}")

        # Fetch product
        try:
            product = fetch_product(barcode)
        except Exception as e:
            print(f"\n[bold red]Error:[/] {e}")
            return

        name = product.get("product_name", "Unnamed product")
        nutr = product.get("nutriments", {})
        serving_size_g = get_serving_size(product)

        print(f"\n[bold green]🍽️  {name.upper()}[/]")
        print(f"[dim]Serving size: {serving_size_g}g[/dim]\n")

        # Use personalized daily calories instead of fixed 2000
        personal_base_caps = BASE_CAPS.copy()
        personal_base_caps['energy-kcal'] = daily_calories

        # Adjust macros proportionally based on calorie needs
        calorie_ratio = daily_calories / 2000
        personal_base_caps['carbohydrates'] = BASE_CAPS['carbohydrates'] * calorie_ratio
        personal_base_caps['fat'] = BASE_CAPS['fat'] * calorie_ratio
        personal_base_caps['proteins'] = BASE_CAPS['proteins'] * calorie_ratio
        personal_base_caps['sugars'] = BASE_CAPS['sugars'] * calorie_ratio

        # Build personalized caps with health conditions
        personal_caps = personal_base_caps.copy()
        for cond, severity in profile.items():
            if cond not in FOCUS or severity == 0:
                continue
            factor = SEVERITY_FACTOR[severity]
            for nut in FOCUS[cond]:
                if nut in personal_caps:
                    personal_caps[nut] = min(personal_caps[nut], personal_base_caps[nut] * factor)

        # Analyze nutrients
        analysis = {}
        for nut, day_cap in personal_caps.items():
            per100 = to_float(nutr.get(f"{nut}_100g"))
            if per100 is None:
                continue
            per_serv = per100 * serving_size_g / 100.0
            pct_dv = per_serv / day_cap * 100
            tot_intake = per_serv * profile["servings"]
            pct_tot = pct_dv * profile["servings"]
            analysis[nut] = {
                "per_serv": per_serv,
                "pct_dv": pct_dv,
                "tot_intake": tot_intake,
                "pct_dv_tot": pct_tot,
            }

        # Calculate health score
        nutri_score = product.get("nutrition_grades", "unknown").upper()
        nova_group = product.get("nova_group", 0)

        # Display results with realistic traffic lights
        def colour(p):
            # More realistic thresholds
            return "green" if p < 15 else "yellow" if p < 30 else "red"

        print(f"[bold cyan]📈 PRODUCT HEALTH METRICS[/]")
        if nutri_score != "UNKNOWN":
            score_color = {"A": "green", "B": "green", "C": "yellow", "D": "red", "E": "red"}.get(nutri_score, "dim")
            print(f" • Nutri-Score: [{score_color}]{nutri_score}[/{score_color}]")

        nova_labels = {1: "Unprocessed/minimally processed",
                       2: "Processed ingredients",
                       3: "Processed foods",
                       4: "Ultra-processed"}
        if nova_group:
            nova_color = "green" if nova_group <= 2 else "yellow" if nova_group == 3 else "red"
            print(f" • NOVA Group: [{nova_color}]{nova_group} - {nova_labels.get(nova_group, 'Unknown')}[/{nova_color}]")

        tbl = Table(
            title=f"Nutrient Impact for {profile['servings']} serving(s)",
            box=rbox.SIMPLE
        )
        tbl.add_column("Nutrient")
        tbl.add_column("Intake", justify="right")
        tbl.add_column("%DV*", justify="right")
        tbl.add_column("Flag", justify="center")

        for nut, d in sorted(analysis.items()):
            flag = colour(d["pct_dv_tot"])
            tint = {"green": "[green]", "yellow": "[yellow]", "red": "[red]"}[flag]
            unit = "kcal" if nut.endswith("kcal") else "g"
            tbl.add_row(
                nut.replace("-", " ").title(),
                f"{d['tot_intake']:.1f} {unit}",
                f"{d['pct_dv_tot']:.0f} %",
                f"{tint}{flag.upper()}[/]"
            )

        print("\n")
        print(tbl)
        print("\n[dim]*% of YOUR personalized daily target[/dim]")

        # Overall verdict
        flag_rank = {"green": 0, "yellow": 1, "red": 2}
        worst_flag = "green"

        for nut, d in analysis.items():
            f = colour(d["pct_dv_tot"])
            max_sev = max(
                [profile.get(c, 0) for c, lst in FOCUS.items() if nut in lst],
                default=0
            )
            if max_sev > 0 and flag_rank[f] > flag_rank[worst_flag]:
                worst_flag = f

        # Consider BMI and weight goal
        calorie_intake = analysis.get('energy-kcal', {}).get('tot_intake', 0)
        if profile['weight_goal'] == 0 and calorie_intake > daily_calories * 0.3:  # Losing weight
            if worst_flag == "green":
                worst_flag = "yellow"

        print(f"\n[bold cyan]🎯 PERSONALIZED VERDICT[/]")
        if worst_flag == "green":
            verdict = "✅  Looks good for your profile and goals. Enjoy!"
        elif worst_flag == "yellow":
            verdict = "🟡  Acceptable in moderation—watch highlighted nutrients for the rest of the day."
        else:
            verdict = "[bold red]⛔  Not recommended[/bold red] — exceeds your personalized targets for your health goals."

        print(verdict)

        # Additional advice based on goals
        if profile['weight_goal'] == 0:  # Losing weight
            print(f"\n[dim]💡 Weight loss tip: This meal is {calorie_intake:.0f} kcal ({(calorie_intake/daily_calories*100):.0f}% of your daily goal)[/dim]")

        print("\n[i dim]This is educational, not medical advice. Consult healthcare professionals for personalized guidance.[/i dim]")

submit_btn.on_click(on_submit_clicked)

#With Gulten-Analysis

In [ ]:

# 🔍 Food Scanner + Personal Health Filter (Colab version)
# ------------------------------------------------------------

import re
import requests
from rich import print, box as rbox
from rich.table import Table
from ipywidgets import widgets, Layout
from IPython.display import display, clear_output

API = "https://world.openfoodfacts.org/api/v0/product/{}.json"

# ------------------------------------------------------------------
# Helper functions
# ------------------------------------------------------------------
def to_float(x):
    """Convert various formats to float, return None if impossible."""
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    cleaned = re.sub(r"[^0-9eE\.\,\+\-]", "", str(x)).replace(",", ".")
    try:
        return float(cleaned)
    except ValueError:
        return None

def get_serving_size(prod, default=100.0):
    """Extract serving size from product data."""
    if prod.get("serving_quantity") is not None:
        v = to_float(prod["serving_quantity"])
        if v:
            return v
    if prod.get("serving_size"):
        m = re.search(r"([\d\.,]+)\s*(g|ml)", prod["serving_size"], re.I)
        if m:
            return to_float(m.group(1))
    return default

def fetch_product(code):
    """Fetch product from OpenFoodFacts API."""
    r = requests.get(API.format(code), timeout=10)
    if r.status_code != 200:
        raise RuntimeError(f"HTTP error {r.status_code}")
    data = r.json()
    if data.get("status") != 1:
        raise ValueError("Product not found.")
    return data["product"]

def analyze_gluten(product):
    """Analyze gluten content from product data."""
    # Check allergens
    allergens = product.get("allergens", "").lower()
    allergens_tags = product.get("allergens_tags", [])

    # Check traces
    traces = product.get("traces", "").lower()
    traces_tags = product.get("traces_tags", [])

    # Check ingredients
    ingredients_text = product.get("ingredients_text", "").lower()

    # Gluten-containing grains
    gluten_keywords = [
        "gluten", "wheat", "barley", "rye", "spelt", "kamut",
        "triticale", "malt", "wheat flour", "wheat starch"
    ]

    # Check for gluten presence
    has_gluten = False
    gluten_sources = []

    for keyword in gluten_keywords:
        if keyword in allergens or keyword in ingredients_text:
            has_gluten = True
            gluten_sources.append(keyword)

    for tag in allergens_tags:
        if "gluten" in tag or "wheat" in tag or "barley" in tag or "rye" in tag:
            has_gluten = True

    # Check for gluten-free label
    labels = product.get("labels", "").lower()
    labels_tags = product.get("labels_tags", [])
    is_gluten_free = "gluten-free" in labels or any("gluten-free" in tag for tag in labels_tags)

    # Check for traces
    has_traces = False
    for keyword in gluten_keywords:
        if keyword in traces:
            has_traces = True

    for tag in traces_tags:
        if "gluten" in tag or "wheat" in tag:
            has_traces = True

    return {
        "has_gluten": has_gluten,
        "gluten_sources": list(set(gluten_sources)),
        "is_gluten_free": is_gluten_free,
        "has_traces": has_traces
    }

# ------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------
QUESTIONS = {
    "diabetes":       "Diabetes",
    "cholesterol":    "High cholesterol",
    "blood_pressure": "High blood pressure",
    "heart":          "Heart condition",
    "celiac":         "Celiac disease/Gluten sensitivity",
    "weight_goal":    "Weight goal",
    "activity":       "Activity level",
}

# Baseline daily caps (realistic adult guidelines)
BASE_CAPS = {
    "sugars":        50,      # WHO: <50g/day (10% of 2000 kcal)
    "carbohydrates": 300,     # ~60% of 2000 kcal
    "fat":           65,      # ~30% of 2000 kcal
    "saturated-fat": 20,      # <10% of 2000 kcal
    "fiber":         30,      # Daily recommendation
    "proteins":      50,      # ~10% of 2000 kcal
    "salt":          5,       # WHO: <5g/day
    "sodium":        2.0,     # WHO: <2g/day (5g salt)
    "cholesterol":   0.3,     # 300mg/day
    "energy-kcal":   2000,    # Average adult
}

# Nutrients that "belong" to each condition
FOCUS = {
    "diabetes":       ["sugars", "carbohydrates"],
    "cholesterol":    ["saturated-fat", "fat", "cholesterol"],
    "blood_pressure": ["salt", "sodium"],
    "heart":          ["saturated-fat", "salt", "cholesterol"],
}

# Severity multipliers: more realistic adjustments
# 0 → normal, 1 → moderately strict (-25%), 2 → very strict (-50%)
SEVERITY_FACTOR = {0: 1.00, 1: 0.75, 2: 0.50}

# ------------------------------------------------------------------
# Create input widgets
# ------------------------------------------------------------------
print("Health Profile Setup")
print("=" * 50)

# Personal info
weight_input = widgets.FloatText(
    value=70.0,
    description='Weight (kg):',
    style={'description_width': '200px'},
    layout=Layout(width='300px')
)

height_input = widgets.FloatText(
    value=170.0,
    description='Height (cm):',
    style={'description_width': '200px'},
    layout=Layout(width='300px')
)

age_input = widgets.IntText(
    value=30,
    description='Age (years):',
    style={'description_width': '200px'},
    layout=Layout(width='300px')
)

gender_input = widgets.RadioButtons(
    options=['Male', 'Female'],
    value='Male',
    description='Gender:',
    style={'description_width': '200px'},
    layout=Layout(width='400px')
)

# Create radio buttons for each condition
widgets_dict = {}
for key, label in QUESTIONS.items():
    if key == "weight_goal":
        widgets_dict[key] = widgets.RadioButtons(
            options=['Lose Weight (0)', 'Maintain (1)', 'Gain Weight (2)'],
            value='Maintain (1)',
            description=label + ':',
            style={'description_width': '200px'},
            layout=Layout(width='500px')
        )
    elif key == "activity":
        widgets_dict[key] = widgets.RadioButtons(
            options=['Sedentary (0)', 'Moderate (1)', 'Very Active (2)'],
            value='Moderate (1)',
            description=label + ':',
            style={'description_width': '200px'},
            layout=Layout(width='500px')
        )
    else:
        widgets_dict[key] = widgets.RadioButtons(
            options=['None (0)', 'Mild/Controlled (1)', 'Severe/Uncontrolled (2)'],
            value='None (0)',
            description=label + ':',
            style={'description_width': '200px'},
            layout=Layout(width='500px')
        )

# Servings input
servings_input = widgets.IntText(
    value=1,
    description='Servings:',
    style={'description_width': '200px'},
    layout=Layout(width='300px')
)

# Barcode input
barcode_input = widgets.Text(
    value='',
    placeholder='Enter barcode...',
    description='Barcode:',
    style={'description_width': '200px'},
    layout=Layout(width='500px')
)

# Submit button
submit_btn = widgets.Button(
    description='Analyze Product',
    button_style='success',
    layout=Layout(width='200px')
)

# Output area
output = widgets.Output()

# Display all widgets
print("\n--- Personal Information ---")
display(weight_input)
display(height_input)
display(age_input)
display(gender_input)

print("\n--- Health & Lifestyle ---")
for w in widgets_dict.values():
    display(w)
display(servings_input)
display(barcode_input)
display(submit_btn)
display(output)

# ------------------------------------------------------------------
# Button click handler
# ------------------------------------------------------------------
def on_submit_clicked(b):
    with output:
        clear_output()

        # Get personal info
        weight = weight_input.value
        height = height_input.value
        age = age_input.value
        gender = gender_input.value

        if weight <= 0 or height <= 0 or age <= 0:
            print("[bold red]Please enter valid personal information![/]")
            return

        # Calculate BMI
        bmi = weight / ((height / 100) ** 2)

        # Calculate BMR (Mifflin-St Jeor Equation)
        if gender == 'Male':
            bmr = (10 * weight) + (6.25 * height) - (5 * age) + 5
        else:
            bmr = (10 * weight) + (6.25 * height) - (5 * age) - 161

        # Get profile values
        profile = {}
        for key, widget in widgets_dict.items():
            val = widget.value
            if '0' in val or 'Sedentary' in val or 'Lose' in val:
                profile[key] = 0
            elif '1' in val or 'Moderate' in val or 'Maintain' in val:
                profile[key] = 1
            else:
                profile[key] = 2

        # Calculate TDEE based on activity level
        activity_multiplier = {0: 1.2, 1: 1.55, 2: 1.9}  # Sedentary, Moderate, Very Active
        tdee = bmr * activity_multiplier[profile['activity']]

        # Adjust calories based on weight goal
        if profile['weight_goal'] == 0:  # Lose weight
            daily_calories = tdee - 500  # 500 calorie deficit
            calorie_advice = "weight loss"
        elif profile['weight_goal'] == 2:  # Gain weight
            daily_calories = tdee + 300  # 300 calorie surplus
            calorie_advice = "weight gain"
        else:  # Maintain
            daily_calories = tdee
            calorie_advice = "maintenance"

        profile["servings"] = servings_input.value
        barcode = barcode_input.value.strip()

        if not barcode:
            print("[bold red]Please enter a barcode![/]")
            return

        if profile["servings"] < 1:
            print("[bold red]Servings must be at least 1![/]")
            return

        # BMI category
        if bmi < 18.5:
            bmi_cat = "Underweight"
            bmi_color = "yellow"
        elif bmi < 25:
            bmi_cat = "Normal"
            bmi_color = "green"
        elif bmi < 30:
            bmi_cat = "Overweight"
            bmi_color = "yellow"
        else:
            bmi_cat = "Obese"
            bmi_color = "red"

        # Display profile
        print("\n[bold cyan]📊 YOUR PROFILE[/]")
        print(f" • Weight: {weight}kg | Height: {height}cm | Age: {age} | Gender: {gender}")
        print(f" • BMI: [{bmi_color}]{bmi:.1f}[/{bmi_color}] ({bmi_cat})")
        print(f" • Daily calories ({calorie_advice}): {daily_calories:.0f} kcal")
        print(f" • Servings to consume: {profile['servings']}")

        print("\n[bold cyan]🏥 HEALTH CONDITIONS[/]")
        levels = {0: "none", 1: "mild", 2: "severe"}
        for k, v in profile.items():
            if k in ["servings", "weight_goal", "activity"]:
                continue
            print(f" • {QUESTIONS[k]}: {levels[v]}")

        # Fetch product
        try:
            product = fetch_product(barcode)
        except Exception as e:
            print(f"\n[bold red]Error:[/] {e}")
            return

        name = product.get("product_name", "Unnamed product")
        nutr = product.get("nutriments", {})
        serving_size_g = get_serving_size(product)

        print(f"\n[bold green]🍽️  {name.upper()}[/]")
        print(f"[dim]Serving size: {serving_size_g}g[/dim]\n")

        # Analyze gluten
        gluten_analysis = analyze_gluten(product)

        print(f"[bold cyan]🌾 GLUTEN ANALYSIS[/]")
        if gluten_analysis["is_gluten_free"]:
            print(" • Status: [bold green]✓ GLUTEN-FREE (Certified)[/bold green]")
            print(" • Gluten content: [green]0%[/green]")
        elif gluten_analysis["has_gluten"]:
            print(" • Status: [bold red]⚠ CONTAINS GLUTEN[/bold red]")
            if gluten_analysis["gluten_sources"]:
                sources = ", ".join(gluten_analysis["gluten_sources"])
                print(f" • Sources: {sources}")
            print(" • Gluten content: [red]Present (exact % not available)[/red]")
            if profile.get("celiac", 0) > 0:
                print(" • [bold red]⛔ NOT SAFE for celiac disease/gluten sensitivity[/bold red]")
        elif gluten_analysis["has_traces"]:
            print(" • Status: [yellow]⚠ MAY CONTAIN TRACES[/yellow]")
            print(" • Gluten content: [yellow]Trace amounts possible[/yellow]")
            if profile.get("celiac", 0) == 2:
                print(" • [yellow]⚠ Use caution if highly sensitive[/yellow]")
        else:
            print(" • Status: [green]✓ No gluten detected[/green]")
            print(" • Gluten content: [green]Not detected[/green]")
            print(" • [dim]Note: Product may not be certified gluten-free[/dim]")

        # Use personalized daily calories instead of fixed 2000
        personal_base_caps = BASE_CAPS.copy()
        personal_base_caps['energy-kcal'] = daily_calories

        # Adjust macros proportionally based on calorie needs
        calorie_ratio = daily_calories / 2000
        personal_base_caps['carbohydrates'] = BASE_CAPS['carbohydrates'] * calorie_ratio
        personal_base_caps['fat'] = BASE_CAPS['fat'] * calorie_ratio
        personal_base_caps['proteins'] = BASE_CAPS['proteins'] * calorie_ratio
        personal_base_caps['sugars'] = BASE_CAPS['sugars'] * calorie_ratio

        # Build personalized caps with health conditions
        personal_caps = personal_base_caps.copy()
        for cond, severity in profile.items():
            if cond not in FOCUS or severity == 0:
                continue
            factor = SEVERITY_FACTOR[severity]
            for nut in FOCUS[cond]:
                if nut in personal_caps:
                    personal_caps[nut] = min(personal_caps[nut], personal_base_caps[nut] * factor)

        # Analyze nutrients
        analysis = {}
        for nut, day_cap in personal_caps.items():
            per100 = to_float(nutr.get(f"{nut}_100g"))
            if per100 is None:
                continue
            per_serv = per100 * serving_size_g / 100.0
            pct_dv = per_serv / day_cap * 100
            tot_intake = per_serv * profile["servings"]
            pct_tot = pct_dv * profile["servings"]
            analysis[nut] = {
                "per_serv": per_serv,
                "pct_dv": pct_dv,
                "tot_intake": tot_intake,
                "pct_dv_tot": pct_tot,
            }

        # Calculate health score
        nutri_score = product.get("nutrition_grades", "unknown").upper()
        nova_group = product.get("nova_group", 0)

        # Display results with realistic traffic lights
        def colour(p):
            # More realistic thresholds
            return "green" if p < 15 else "yellow" if p < 30 else "red"

        print(f"\n[bold cyan]📈 PRODUCT HEALTH METRICS[/]")
        if nutri_score != "UNKNOWN":
            score_color = {"A": "green", "B": "green", "C": "yellow", "D": "red", "E": "red"}.get(nutri_score, "dim")
            print(f" • Nutri-Score: [{score_color}]{nutri_score}[/{score_color}]")

        nova_labels = {1: "Unprocessed/minimally processed",
                       2: "Processed ingredients",
                       3: "Processed foods",
                       4: "Ultra-processed"}
        if nova_group:
            nova_color = "green" if nova_group <= 2 else "yellow" if nova_group == 3 else "red"
            print(f" • NOVA Group: [{nova_color}]{nova_group} - {nova_labels.get(nova_group, 'Unknown')}[/{nova_color}]")

        tbl = Table(
            title=f"Nutrient Impact for {profile['servings']} serving(s)",
            box=rbox.SIMPLE
        )
        tbl.add_column("Nutrient")
        tbl.add_column("Intake", justify="right")
        tbl.add_column("%DV*", justify="right")
        tbl.add_column("Flag", justify="center")

        for nut, d in sorted(analysis.items()):
            flag = colour(d["pct_dv_tot"])
            tint = {"green": "[green]", "yellow": "[yellow]", "red": "[red]"}[flag]
            unit = "kcal" if nut.endswith("kcal") else "g"
            tbl.add_row(
                nut.replace("-", " ").title(),
                f"{d['tot_intake']:.1f} {unit}",
                f"{d['pct_dv_tot']:.0f} %",
                f"{tint}{flag.upper()}[/]"
            )

        print("\n")
        print(tbl)
        print("\n[dim]*% of YOUR personalized daily target[/dim]")

        # Overall verdict - consider gluten status
        flag_rank = {"green": 0, "yellow": 1, "red": 2}
        worst_flag = "green"

        for nut, d in analysis.items():
            f = colour(d["pct_dv_tot"])
            max_sev = max(
                [profile.get(c, 0) for c, lst in FOCUS.items() if nut in lst],
                default=0
            )
            if max_sev > 0 and flag_rank[f] > flag_rank[worst_flag]:
                worst_flag = f

        # Check gluten for celiac condition
        if profile.get("celiac", 0) > 0 and gluten_analysis["has_gluten"]:
            worst_flag = "red"
        elif profile.get("celiac", 0) == 2 and gluten_analysis["has_traces"]:
            if worst_flag != "red":
                worst_flag = "yellow"

        # Consider BMI and weight goal
        calorie_intake = analysis.get('energy-kcal', {}).get('tot_intake', 0)
        if profile['weight_goal'] == 0 and calorie_intake > daily_calories * 0.3:  # Losing weight
            if worst_flag == "green":
                worst_flag = "yellow"

        print(f"\n[bold cyan]🎯 PERSONALIZED VERDICT[/]")
        if worst_flag == "green":
            verdict = "✅  Looks good for your profile and goals. Enjoy!"
        elif worst_flag == "yellow":
            verdict = "🟡  Acceptable in moderation—watch highlighted nutrients for the rest of the day."
        else:
            verdict = "[bold red]⛔  Not recommended[/bold red] — exceeds your personalized targets for your health goals."

        print(verdict)

        # Additional advice based on goals
        if profile['weight_goal'] == 0:  # Losing weight
            print(f"\n[dim]💡 Weight loss tip: This meal is {calorie_intake:.0f} kcal ({(calorie_intake/daily_calories*100):.0f}% of your daily goal)[/dim]")

        print("\n[i dim]This is educational, not medical advice. Consult healthcare professionals for personalized guidance.[/i dim]")

submit_btn.on_click(on_submit_clicked)